In [2]:
from lib2to3.pgen2 import token
import re
from flask import Flask,render_template,url_for,request
import pandas as pd 
import numpy as np
from scipy.special import softmax
import os  
import random
import torch

import config
from utils import *
from processor import *

import stanza
stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize')

from transformers import BertForTokenClassification, BertTokenizer
num_labels = len(tag2idx)
save_model_address = './trained_models/C-Bert-test'
model = BertForTokenClassification.from_pretrained(save_model_address, num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(save_model_address, do_lower_case=False)

2022-04-18 09:16:38 INFO: Downloading default packages for language: en (English)...
2022-04-18 09:16:39 INFO: File exists: /Users/administrator/stanza_resources/en/default.zip.
2022-04-18 09:16:42 INFO: Finished downloading models and saved to /Users/administrator/stanza_resources.
2022-04-18 09:16:42 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-04-18 09:16:42 INFO: Use device: cpu
2022-04-18 09:16:42 INFO: Loading: tokenize
2022-04-18 09:16:42 INFO: Done loading processors!


In [11]:
long_text = "ALLERGIES:  Norvasc leads to lightheadedness and headache.  FAMILY HISTORY:  Noncontributory.  SOCIAL HISTORY:  Lives with her husband, Dr. [**Known lastname 1809**] an eminent Pediatric Neurologist at [**Hospital3 1810**].  The patient is a prior smoker, but has not smoked in over 10 years.  She has no known alcohol use and she is a full code."

In [24]:
doc = nlp(long_text)
all_sentences = []
all_tags = []
for no, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {no + 1} tokens =======')
    # temp_token: tokenized words
    # input_ids: convert temp_token to id
    temp_token, input_ids, attention_masks = create_query(sentence, tokenizer)
    result_list = model_inference(model, input_ids)
    result = [tag2name[t] for t in result_list]
    pretok_sent = ""
    pretags = ""
    for i, tok in enumerate(temp_token):
        if tok.startswith("##"):
            pretok_sent += tok[2:]
        else:
            pretok_sent += " " + tok
            pretags += " " + result[i]
    pretok_sent = pretok_sent[1:]
    pretags = pretags[1:]
    s = pretok_sent.split()
    t = pretags.split()
    all_sentences.append(s)
    all_tags.append(t)

====== Sentence 1 tokens =======
====== Sentence 2 tokens =======
====== Sentence 3 tokens =======
====== Sentence 4 tokens =======
====== Sentence 5 tokens =======


In [33]:
for s, t in zip(all_sentences[2], all_tags[2]):
    print('word:', s, end='   ')
    print('tag:', t)

word: [CLS]   tag: [CLS]
word: SOCIAL   tag: O
word: HISTORY   tag: O
word: :   tag: O
word: Lives   tag: O
word: with   tag: O
word: her   tag: O
word: husband   tag: O
word: ,   tag: O
word: Dr   tag: O
word: .   tag: X
word: [   tag: O
word: *   tag: X
word: *   tag: X
word: Known   tag: O
word: lastname   tag: O
word: 1809   tag: O
word: *   tag: X
word: *   tag: X
word: ]   tag: X
word: an   tag: O
word: eminent   tag: O
word: Pediatric   tag: O
word: Neurologist   tag: O
word: at   tag: O
word: [   tag: O
word: *   tag: X
word: *   tag: X
word: Hospital3   tag: O
word: 1810   tag: O
word: *   tag: X
word: *   tag: X
word: ]   tag: X
word: .   tag: O
word: [SEP]   tag: [SEP]


In [ ]:
for s, t in zip(all_sentences, all_tags):
    flag_treatment, flag_problem, flag_test = 0, 0, 0
    for i in range(1, len(t)):
        print()

In [27]:
def print_treatment(word):
    return colored(word,'white','on_red', attrs=['underline'])
def print_test(word):
    return colored(word, 'white','on_magenta')
def print_problem(word):
    return colored(word, 'magenta','on_cyan', attrs=['reverse', 'bold'])

In [34]:
location = {}
for s, t in zip(all_sentences, all_tags):
    flag_treatment, flag_problem, flag_test = 0, 0, 0
    for i in range(1, len(t)):
        if t[i] == 'B-treatment':
            flag_treatment = 1
            print(print_treatment(s[i]), end=' ')
        elif t[i] == 'I-treatment' or t[i] == 'X' and flag_treatment == 1 :
            print(print_treatment(s[i]), end=' ')
        elif t[i] == 'B-test':
            flag_test = 1
            print(print_test(s[i]), end=' ')
        elif t[i] == 'I-test' or t[i] == 'X' and flag_test == 1 :
            print(print_test(s[i]), end=' ')
        elif t[i] == 'B-problem':
            flag_problem = 1
            print(print_problem(s[i]), end=' ')
        elif t[i] == 'I-problem' or t[i] == 'X' and flag_problem == 1 :
            print(print_problem(s[i]), end=' ')
        elif t[i] == 'O' or t[i] == 'X':
            flag_treatment, flag_problem, flag_test = 0, 0, 0
            print(s[i], end=' ')

ALLERGIES : Norvasc leads to lightheadedness and headache . FAMILY HISTORY : Noncontributory . SOCIAL HISTORY : Lives with her husband , Dr . [ * * Known lastname 1809 * * ] an eminent Pediatric Neurologist at [ * * Hospital3 1810 * * ] . The patient is a prior smoker , but has not smoked in over 10 years . She has no known alcohol use and she is a full code . 

In [14]:
s

['[CLS]',
 'She',
 'has',
 'no',
 'known',
 'alcohol',
 'use',
 'and',
 'she',
 'is',
 'a',
 'full',
 'code',
 '.',
 '[SEP]']

In [3]:
temp_token, input_ids, attention_masks = create_query(sentence, tokenizer)

AttributeError: 'str' object has no attribute 'tokens'

In [3]:
message = """HISTORY OF PRESENT ILLNESS :
The patient is an 80 year old female with breast cancer , status post lumpectomy / radiation therapy / Tamoxifen ( 2000 ) , hypertension , hyperlipidemia , multiple urinary tract infections who presents with a four day prodrome of dry cough , rhinorrhea , coryza , malaise , chills , headache , decreased p.o. intake , loose bowel movements with diarrhea and no blood , decreased urine output , no sick contacts , had flu shot this year .
In the Emergency Department , she had labile blood pressure with systolic blood pressure in the 80 s .  
Her usual is systolic blood pressure in the 120 s , this was despite two liters of intravenous fluids and she was transferred to the Medical Intensive Care Unit for closer monitoring for possible early sepsis .
The Intensive Care Unit course was notable for initially receiving broad spectrum antibiotics as well as a white blood cell count of 11.0 with 15 bands .
She was ultimately changed to Levaquin for a possible early pneumonia pending cultures .
The Intensive Care Unit course was also notable for negative chest x-ray , two units of packed red blood cells for a hematocrit of 24.0 with appropriate bump in her hematocrit and no evidence of bleeding , stable blood pressure despite a net fluid balance of negative 2.5 liters .
No central access was needed .
The sepsis protocol was aborted ."""

In [4]:
for anno in sd_model.fullAnnotate(message)[0]["sentences"]:
    print(anno)

Annotation(document, 0, 27, HISTORY OF PRESENT ILLNESS :, {'sentence': '0'})
Annotation(document, 29, 466, The patient is an 80 year old female with breast cancer , status post lumpectomy / radiation therapy / Tamoxifen ( 2000 ) , hypertension , hyperlipidemia , multiple urinary tract infections who presents with a four day prodrome of dry cough , rhinorrhea , coryza , malaise , chills , headache , decreased p.o. intake , loose bowel movements with diarrhea and no blood , decreased urine output , no sick contacts , had flu shot this year ., {'sentence': '1'})
Annotation(document, 468, 569, In the Emergency Department , she had labile blood pressure with systolic blood pressure in the 80 s ., {'sentence': '2'})
Annotation(document, 573, 782, Her usual is systolic blood pressure in the 120 s , this was despite two liters of intravenous fluids and she was transferred to the Medical Intensive Care Unit for closer monitoring for possible early sepsis ., {'sentence': '3'})
Annotation(documen

In [8]:
import math
import pandas as pd
import numpy as np
import os

from scipy.special import softmax
from tqdm import trange
import torch

from scipy.special import softmax
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertForTokenClassification, BertTokenizer


In [9]:
clinical_text = [['He',
 'was',
 'admitted',
 ',',
 'taken',
 'to',
 'the',
 'operating',
 'room',
 'where',
 'he',
 'underwent',
 'L5-S1',
 'right',
 'hemilaminectomy',
 'and',
 'discectomy',
 '.']
, 
['Over',
 'the',
 'next',
 'three',
 'days',
 'he',
 'increased',
 'his',
 'activity',
 'gradually',
 ',',
 'was',
 'able',
 'to',
 'do',
 'stairs',
 'with',
 'Physical',
 'Therapy',
 'and',
 'had',
 'pain',
 'which',
 'could',
 'be',
 'controlled',
 'with',
 'oral',
 'analgesics',
 '.'],

['However', ',', 'her', 'creatinine', 'continued', 'to', 'increase', '.']
]

In [10]:
clinical_tags = [['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-treatment',
 'I-treatment',
 'I-treatment',
 'O',
 'B-treatment',
 'O'],
                 
 ['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-treatment',
 'I-treatment',
 'O',
 'O',
 'B-problem',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-treatment',
 'I-treatment',
 'O'],
 
 ['O', 'O', 'B-test', 'I-test', 'O', 'O', 'O', 'O']                
]


In [11]:

bert_out_address = './trained_models/C-Bert-test'

tag2idx={'B-problem': 0,
 'B-test': 1,
 'B-treatment': 2,
 'I-problem': 3,
 'I-test': 4,
 'I-treatment': 5,
 'O': 6,
 'X': 7,
 '[CLS]': 8,
 '[SEP]': 9
 }

tag2name = {0: 'B-problem',
    1: 'B-test',
    2: 'B-treatment',
    3: 'I-problem',
    4: 'I-test',
    5: 'I-treatment',
    6: 'O',
    7: 'X',
    8: '[CLS]',
    9: '[SEP]'}

max_len  = 128

In [12]:
test_query = clinical_text[0]

In [13]:
model = BertForTokenClassification.from_pretrained(bert_out_address, num_labels=len(tag2idx))
tokenizer = BertTokenizer.from_pretrained(bert_out_address, do_lower_case=False)


In [14]:
test_query = ' '.join(test_query)


In [15]:
test_query

'He was admitted , taken to the operating room where he underwent L5-S1 right hemilaminectomy and discectomy .'

In [16]:
tokenized_texts = []
temp_token = []
temp_token.append('[CLS]')
token_list = tokenizer.tokenize(test_query)
temp_token.extend(token_list)
temp_token = temp_token[:max_len-1]
temp_token.append('[SEP]')
input_id = tokenizer.convert_tokens_to_ids(temp_token)
padding_len = max_len - len(input_id)
input_id = input_id + ([0] * padding_len)
tokenized_texts.append(input_id)
attention_masks = [[int(i>0) for i in input_id]]

tokenized_texts = torch.tensor(tokenized_texts)
attention_masks = torch.tensor(attention_masks)

In [17]:
tokenized_texts

tensor([[  101,  1124,  1108,  4120,   117,  1678,  1106,  1103,  3389,  1395,
          1187,  1119,  9315,   149,  1571,   118,   156,  1475,  1268, 23123,
          8009,  9685,  5822, 18574,  1105,  6187, 10294, 18778,  1183,   119,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [28]:
# Set save model to Evalue loop
model.eval()
# Get model predict result
with torch.no_grad():
        outputs = model(tokenized_texts, token_type_ids=None,
        attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 

In [29]:
predict_results = logits.detach().cpu().numpy()
result_arrays_soft = softmax(predict_results[0])
result_list = np.argmax(result_arrays_soft,axis=-1)

In [30]:
result = [tag2name[t] for t in result_list]
pretok_sent = ""
pretags = ""
for i, tok in enumerate(temp_token):
     if tok.startswith("##"):
         pretok_sent += tok[2:]
     else:
         pretok_sent += " " + tok
         pretags += " " + result[i]
pretok_sent = pretok_sent[1:]
pretags = pretags[1:]

s = pretok_sent.split()
t = pretags.split()

In [31]:
for i, mark in enumerate(attention_masks[0]):
    if mark>0:
        print("Token:%s"%(temp_token[i]))
#         print("Tag:%s"%(result_list[i]))
        print("Predict_Tag:%s"%(tag2name[result_list[i]]))
        #print("Posibility:%f"%(result_array[i][result_list[i]]))
        print()
        

Token:[CLS]
Predict_Tag:O

Token:He
Predict_Tag:O

Token:was
Predict_Tag:O

Token:admitted
Predict_Tag:O

Token:,
Predict_Tag:O

Token:taken
Predict_Tag:O

Token:to
Predict_Tag:O

Token:the
Predict_Tag:O

Token:operating
Predict_Tag:O

Token:room
Predict_Tag:O

Token:where
Predict_Tag:O

Token:he
Predict_Tag:O

Token:underwent
Predict_Tag:O

Token:L
Predict_Tag:B-treatment

Token:##5
Predict_Tag:X

Token:-
Predict_Tag:X

Token:S
Predict_Tag:X

Token:##1
Predict_Tag:X

Token:right
Predict_Tag:I-treatment

Token:hem
Predict_Tag:I-treatment

Token:##ila
Predict_Tag:X

Token:##mine
Predict_Tag:X

Token:##ct
Predict_Tag:X

Token:##omy
Predict_Tag:X

Token:and
Predict_Tag:O

Token:disc
Predict_Tag:B-treatment

Token:##ec
Predict_Tag:X

Token:##tom
Predict_Tag:X

Token:##y
Predict_Tag:X

Token:.
Predict_Tag:O

Token:[SEP]
Predict_Tag:[SEP]



In [32]:
text = ' '.join(temp_token)
re = ' '.join(result)

In [33]:
import docx
from docx.enum.text import WD_COLOR_INDEX
# Create an instance of a word document
doc = docx.Document()

# Add a Title to the document 
doc.add_heading('Results', 0)

# Creating paragraph with some content
para = doc.add_paragraph(''' ''')
  
flag_treatment, flag_problem, flag_test = 0, 0, 0 
for i in range(1, len(t)-1):
    if t[i] == 'B-treatment':
        flag_treatment = 1
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.RED
        # print(print_treatment(s[i]), end=' ')
    elif (t[i] == 'I-treatment' or t[i] == 'X') and flag_treatment == 1 :
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.RED
    elif t[i] == 'B-test':
        flag_test = 1
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.PINK
    elif (t[i] == 'I-test' or t[i] == 'X') and flag_test == 1 :
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.PINK
    elif t[i] == 'B-problem':
        flag_problem = 1
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.TURQUOISE
    elif (t[i] == 'I-problem' or t[i] == 'X') and flag_problem == 1 :
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.TURQUOISE    
    elif t[i] == 'O':
        flag_treatment, flag_problem, flag_test = 0, 0, 0 
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.AUTO
        

# # Adding more content to paragraph and highlighting them
# para.add_run(''' It contains well written, well thought and well-explained '''
#             ).font.highlight_color = WD_COLOR_INDEX.YELLOW
  
# # Adding more content to paragraph
# para.add_run('''computer science and programming articles, quizzes etc.''')
  
# Now save the document to a location 
doc.save('result.docx')


In [34]:
from termcolor import colored
def print_treatment(word):
    return colored(word,'white','on_red', attrs=['underline'])
def print_test(word):
    return colored(word, 'white','on_magenta')
def print_problem(word):
    return colored(word, 'magenta','on_cyan', attrs=['reverse', 'bold'])


In [35]:
print(print_problem('problem'))
print(print_treatment('treatment'))
print(print_test('test'))

problem
treatment
test


In [36]:
from termcolor import colored
location = {}
flag_treatment, flag_problem, flag_test = 0, 0, 0 
for i in range(1, len(t)-1):
    if t[i] == 'B-treatment':
        flag_treatment = 1
        print(print_treatment(s[i]), end=' ')
    elif t[i] == 'I-treatment' or t[i] == 'X' and flag_treatment == 1 :
        print(print_treatment(s[i]), end=' ')
    elif t[i] == 'B-test':
        flag_test = 1
        print(print_test(s[i]), end=' ')
    elif t[i] == 'I-test' or t[i] == 'X' and flag_test == 1 :
        print(print_test(s[i]), end=' ')
    elif t[i] == 'B-problem':
        flag_problem = 1
        print(print_problem(s[i]), end=' ')
    elif t[i] == 'I-problem' or t[i] == 'X' and flag_problem == 1 :
        print(print_problem(s[i]), end=' ')    
    elif t[i] == 'O':
        flag_treatment, flag_problem, flag_test = 0, 0, 0 
        print(s[i], end=' ')

He was admitted , taken to the operating room where he underwent L5 - S1 right hemilaminectomy and discectomy . 

In [ ]:
temp_lable = []
temp_token = []

# Add [CLS] at the front 
temp_lable.append('[CLS]')
temp_token.append('[CLS]')

# Tokenize words
# [lidocaine patch] -> lid ##oc ##aine patch
# [B-treatment I-treatment] -> B-treatment X X I-treatment
for word, lab in zip(word_list, label):
    token_list = tokenizer.tokenize(word)
    for m, token in enumerate(token_list):
        temp_token.append(token)
        if m == 0:
            temp_lable.append(lab)
        else:
            temp_lable.append('X')  
            
# Add [SEP] at the end
temp_lable.append('[SEP]')
temp_token.append('[SEP]')

In [5]:
class SentenceGetter(object) :
    """
    input a dataframe
    
    
    Generate sets of words and tags.
    self.sentence:
    Each sentence is a list
    [('Supraventricular', 'B-problem'),
    ('tachycardia', 'I-problem'),
    ('(', 'O'),
    ('on', 'O'),
    ('a', 'B-treatment'),
    ('beta', 'I-treatment'),
    ('blocker', 'I-treatment'),
    (')', 'O')]
    
    """
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),
                                                        #    s["POS"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
    
    def get_examples(self):
        return random.sample(self.sentences, 10)
    
    def get_sentences(self):
        return[[s[0] for s in sent] for sent in self.sentences]
        
    def get_labels(self):
        return [[s[1] for s in sent] for sent in self.sentences]

In [6]:
class InputGenerater(object):
    def __init__(self, sentences, labels) -> None:
        self.tokenized_texts = []
        self.word_piece_labels = []
        i_inc = 0
        for word_list,label in (zip(sentences,labels)):
            temp_lable = []
            temp_token = []
            
            # Add [CLS] at the front 
            temp_lable.append('[CLS]')
            temp_token.append('[CLS]')
            
            for word,lab in zip(word_list,label):
                token_list = tokenizer.tokenize(word)
                for m,token in enumerate(token_list):
                    temp_token.append(token)
                    if m==0:
                        temp_lable.append(lab)
                    else:
                        temp_lable.append('X')  
                        
            # Add [SEP] at the end
            temp_lable.append('[SEP]')
            temp_token.append('[SEP]')
            
            self.tokenized_texts.append(temp_token)
            self.word_piece_labels.append(temp_lable)
            
            if 5 > i_inc:
                print("No.%d,len:%d"%(i_inc,len(temp_token)))
                print("texts:%s"%(" ".join(temp_token)))
                print("No.%d,len:%d"%(i_inc,len(temp_lable)))
                print("lables:%s"%(" ".join(temp_lable)))
            i_inc +=1
            
        self.input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in self.tokenized_texts],
                    maxlen=config.MAX_LEN, dtype="long", truncating="post", padding="post")
            
    def get_input_ids(self):    
        return self.input_ids
    
    def get_tags(self):
        tags = pad_sequences([[get_tag2idx().get(l) for l in lab] for lab in self.word_piece_labels],
                        maxlen=config.MAX_LEN, value=get_tag2idx()["O"], padding="post",
                        dtype="long", truncating="post")
        return tags
    def get_attention_masks(self):
        attention_masks = [[int(i>0) for i in ii] for ii in self.input_ids]
        return attention_masks
    def get_segment_ids(self):
        segment_ids = [[0] * len(input_id) for input_id in self.input_ids]
        return segment_ids


def convert_to_tensor(*inputs, drop_last=False):
    data = TensorDataset(*tuple(torch.tensor(inputs)))
    data_sampler = RandomSampler(data)
    # Drop last can make batch training better for the last one
    dataloader = DataLoader(data, sampler=data_sampler, batch_size=config.BATCH_NUM, drop_last=drop_last)
    return dataloader

In [10]:
df_test = pd.read_csv(config.data_path_test, sep="\t").astype(str)
sg = SentenceGetter(df_test)
sentences = sg.get_sentences()
tags = sg.get_labels()
test_sets = InputGenerater(sentences=sentences, labels=tags)

test_inputs = test_sets.get_input_ids()
test_tags = test_sets.get_tags()
test_attetion_masks = test_sets.get_attention_masks()

test_dataloader = utils.convert_to_tensor(test_inputs, test_attetion_masks, test_tags)


No.0,len:5
texts:[CLS] admission date : [SEP]
No.0,len:5
lables:[CLS] O O O [SEP]
No.1,len:7
texts:[CLS] 2014 - 12 - 29 [SEP]
No.1,len:7
lables:[CLS] O X X X X [SEP]
No.2,len:6
texts:[CLS] all ##er ##gies : [SEP]
No.2,len:6
lables:[CLS] O X X O [SEP]
No.3,len:4
texts:[CLS] 17 units [SEP]
No.3,len:4
lables:[CLS] O O [SEP]
No.4,len:22
texts:[CLS] includes a history of at ##rial fi ##bri ##lla ##tion with good heart rate control on dig ##ox ##in . [SEP]
No.4,len:22
lables:[CLS] O O O O B-problem X I-problem X X X O O B-treatment I-treatment I-treatment O B-treatment X X O [SEP]


NameError: name 'pad_sequences' is not defined